# Content-based Filtering

Aproach:
- Table mit Features und Ratings erstellen, ohne User - auf Basis des OMDB Datasets

- sklearn-kNN mit cosine-similarity darauf anwenden

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

## This is a WIP: Decided to do omdb data cleaning first - bevore a useful model can be applied

Sources:

#### https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

#### https://www.kaggle.com/johnwill225/movie-recommendations

#### https://towardsdatascience.com/how-we-built-a-content-based-filtering-recommender-system-for-music-with-python-c6c3b1020332


In [34]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [35]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_relevant.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [36]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [37]:
ratings.head()

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0


In [38]:
omdb.head()

,Year,Rated,Released,Runtime,imdbRating,imdbVotes,imdbID,Rotten Tomatoes,Metacritic,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,1995.0,0.0,11.0,81.0,8.3,852896.0,tt0114709,10.0,9.5,NaN,3.0,NaN,NaN
1,1995.0,1.0,12.0,104.0,7.0,294340.0,tt0113497,5.4,3.9,NaN,NaN,NaN,NaN
2,1993.0,2.0,12.0,103.0,7.0,41401.0,tt0107050,6.3,5.3,NaN,NaN,NaN,NaN
3,1995.0,3.0,12.0,124.0,5.9,9222.0,tt0114885,5.6,NaN,NaN,NaN,NaN,NaN
4,1995.0,1.0,12.0,106.0,6.0,33005.0,tt0113041,4.8,4.9,NaN,NaN,NaN,1.0


In [39]:
omdb.isna().sum()

Year                          0
Rated                      2420
Released                     58
Runtime                       9
imdbRating                    4
imdbVotes                     7
imdbID                        0
Rotten Tomatoes            1297
Metacritic                 4177
Oscars_won                 9497
Oscars_nominated           9037
Golden_globe_won          10132
Golden_globe_nominated     9840
dtype: int64

In [40]:
# Replace NaN with 0 and handle accordingly for awards
for i in range (9,13):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(0)

In [41]:
omdb.isna().sum()

Year                         0
Rated                     2420
Released                    58
Runtime                      9
imdbRating                   4
imdbVotes                    7
imdbID                       0
Rotten Tomatoes           1297
Metacritic                4177
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [42]:
#Shuffling and then restricting data here to save runtime -> TODO: Find better/faster solution
#ratings = ratings.sample(axis=0,frac=1)
#restricted_ratings = ratings.iloc[:10192]

In [43]:
merged_data = omdb.merge(ratings, how='left', on='imdbID')

In [44]:
merged_data.head()

,Year,Rated,Released,Runtime,imdbRating,imdbVotes,imdbID,Rotten Tomatoes,Metacritic,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,Unnamed: 0,user_id,rating
0,1995.0,0.0,11.0,81.0,8.3,852896.0,tt0114709,10.0,9.5,0.0,3.0,0.0,0.0,394.0,1198.0,3.5
1,1995.0,0.0,11.0,81.0,8.3,852896.0,tt0114709,10.0,9.5,0.0,3.0,0.0,0.0,1171.0,551.0,3.5
2,1995.0,0.0,11.0,81.0,8.3,852896.0,tt0114709,10.0,9.5,0.0,3.0,0.0,0.0,3682.0,412.0,2.5
3,1995.0,0.0,11.0,81.0,8.3,852896.0,tt0114709,10.0,9.5,0.0,3.0,0.0,0.0,5773.0,1087.0,3.5
4,1995.0,0.0,11.0,81.0,8.3,852896.0,tt0114709,10.0,9.5,0.0,3.0,0.0,0.0,7172.0,344.0,4.0


In [45]:
merged_data = merged_data.drop(columns={'Unnamed: 0'})

In [46]:
#Drop NaN rows - check later if it makes large difference!
merged_data = merged_data.dropna()

In [47]:
merged_data = merged_data.set_index('imdbID').reset_index()


In [48]:
#TODO restrict for feasability

In [49]:
data_as_array = merged_data.to_numpy()

In [50]:
le = preprocessing.LabelEncoder()
for i in range(0,13):
   data_as_array[:,i] = le.fit_transform(data_as_array[:,i])

In [51]:
csr_data = csr_matrix(data_as_array.astype(np.float))
merged_data.reset_index(inplace=True)

In [52]:
k_neighbors = 10
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

In [53]:
print("Training")
knn.fit(csr_data)

Training


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [54]:
csr_data

<813270x15 sparse matrix of type '<class 'numpy.float64'>'
	with 9278944 stored elements in Compressed Sparse Row format>

In [55]:
# Do predictions
print("Do Prediction")
movie_idx = merged_data[merged_data['imdbID'] == 'tt0113497'].index[0]
print(movie_idx)
distances, indices = knn.kneighbors(csr_data[movie_idx], n_neighbors=k_neighbors+1)
print(distances)
print(indices)

Do Prediction
1185
[[0.00000000e+00 3.04953465e-08 9.75817618e-08 9.75833618e-08
  2.25663345e-07 3.96422345e-07 4.45230603e-07 6.34273547e-07
  8.84362426e-07 1.58566605e-06 1.58576912e-06]]
[[1185 1755 1388 1587 1503 1494 1829 1562 1195 1757 1251]]


In [56]:
for i in range(0,indices.size):
    print(merged_data.iloc[[indices[0,i]]])

      index     imdbID    Year  Rated  Released  Runtime  imdbRating  \
1185   1185  tt0113497  1995.0    1.0      12.0    104.0         7.0   

      imdbVotes  Rotten Tomatoes  Metacritic  Oscars_won  Oscars_nominated  \
1185   294340.0              5.4         3.9         0.0               0.0   

      Golden_globe_won  Golden_globe_nominated  user_id  rating  
1185               0.0                     0.0     42.0     3.0  
      index     imdbID    Year  Rated  Released  Runtime  imdbRating  \
1755   1755  tt0113497  1995.0    1.0      12.0    104.0         7.0   

      imdbVotes  Rotten Tomatoes  Metacritic  Oscars_won  Oscars_nominated  \
1755   294340.0              5.4         3.9         0.0               0.0   

      Golden_globe_won  Golden_globe_nominated  user_id  rating  
1755               0.0                     0.0     41.0     3.5  
      index     imdbID    Year  Rated  Released  Runtime  imdbRating  \
1388   1388  tt0113497  1995.0    1.0      12.0    104.0    

In [57]:
# zip bildet tuple der beiden Objekte)
ec_movie = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
  

In [58]:
for i in ec_movie:
    print(i)

(1251, 1.585769116574376e-06)
(1757, 1.5856660521285093e-06)
(1195, 8.843624256238414e-07)
(1562, 6.34273547239772e-07)
(1829, 4.4523060327428254e-07)
(1494, 3.9642234528347586e-07)
(1503, 2.256633453434631e-07)
(1587, 9.758336183551819e-08)
(1388, 9.75817617820951e-08)
(1755, 3.049534647026064e-08)


In [59]:
 user_ratings = merged_data.loc[merged_data['user_id'] == 123]

In [60]:
user_ratings

,index,imdbID,Year,Rated,Released,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,user_id,rating
5721,5721,tt0112281,1995.0,2.0,11.0,90.0,6.4,196970.0,3.1,4.5,0.0,0.0,0.0,0.0,123.0,1.5
6693,6693,tt0113627,1995.0,3.0,2.0,111.0,7.5,111148.0,9.0,8.2,1.0,0.0,0.0,0.0,123.0,5.0
7801,7801,tt0114746,1995.0,3.0,1.0,129.0,8.0,563295.0,9.0,7.4,0.0,2.0,0.0,0.0,123.0,4.0
17242,17242,tt0075314,1976.0,3.0,2.0,114.0,8.3,691354.0,9.5,9.4,0.0,4.0,0.0,0.0,123.0,5.0
18110,18110,tt0145653,1999.0,3.0,1.0,145.0,7.3,19722.0,5.2,5.4,0.0,1.0,0.0,0.0,123.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700813,700813,tt0212712,2004.0,3.0,9.0,129.0,7.4,49079.0,8.6,7.8,0.0,0.0,0.0,0.0,123.0,5.0
705597,705597,tt0395169,2004.0,2.0,2.0,121.0,8.1,325743.0,9.1,7.9,0.0,3.0,0.0,0.0,123.0,4.5
741296,741296,tt0075314,1976.0,3.0,2.0,114.0,8.3,691354.0,9.5,9.4,0.0,4.0,0.0,0.0,123.0,5.0
744482,744482,tt0407265,2005.0,3.0,3.0,103.0,7.4,39679.0,7.7,6.6,0.0,2.0,0.0,0.0,123.0,5.0


In [61]:
user_ratings['imdbID'] = user_ratings['imdbID'].str.replace(r'tt','')

<ipython-input-61-f7fe76eb0358>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings['imdbID'] = user_ratings['imdbID'].str.replace(r'tt','')


In [62]:
user_ratings['imdbID'] = user_ratings['imdbID'].astype(float)

<ipython-input-62-f96a71993919>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_ratings['imdbID'] = user_ratings['imdbID'].astype(float)


In [63]:
user_ratings

,index,imdbID,Year,Rated,Released,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,user_id,rating
5721,5721,112281.0,1995.0,2.0,11.0,90.0,6.4,196970.0,3.1,4.5,0.0,0.0,0.0,0.0,123.0,1.5
6693,6693,113627.0,1995.0,3.0,2.0,111.0,7.5,111148.0,9.0,8.2,1.0,0.0,0.0,0.0,123.0,5.0
7801,7801,114746.0,1995.0,3.0,1.0,129.0,8.0,563295.0,9.0,7.4,0.0,2.0,0.0,0.0,123.0,4.0
17242,17242,75314.0,1976.0,3.0,2.0,114.0,8.3,691354.0,9.5,9.4,0.0,4.0,0.0,0.0,123.0,5.0
18110,18110,145653.0,1999.0,3.0,1.0,145.0,7.3,19722.0,5.2,5.4,0.0,1.0,0.0,0.0,123.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700813,700813,212712.0,2004.0,3.0,9.0,129.0,7.4,49079.0,8.6,7.8,0.0,0.0,0.0,0.0,123.0,5.0
705597,705597,395169.0,2004.0,2.0,2.0,121.0,8.1,325743.0,9.1,7.9,0.0,3.0,0.0,0.0,123.0,4.5
741296,741296,75314.0,1976.0,3.0,2.0,114.0,8.3,691354.0,9.5,9.4,0.0,4.0,0.0,0.0,123.0,5.0
744482,744482,407265.0,2005.0,3.0,3.0,103.0,7.4,39679.0,7.7,6.6,0.0,2.0,0.0,0.0,123.0,5.0


In [64]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors
## Code works only if no movie was rated more than once by a certain user

def predict_movie_rating(imdbID, userID, user_data):
    # get 20 (or x) nearest neighbors that were rated by userID
    # TODO: Use function from kNN.ipynb to find optimal k for algorithm 
    
    k_neighbors = 20    
    
    # Select all ratings given by User #userID
    ratings = user_data.loc[user_data['user_id'] == userID]
    print(user_data)
    # Get real rating -> remove this in the end -> currently done for validation
    real_ratings = ratings.loc[ratings['imdbID'] == imdbID]
    
    print('Real Rating: '+str(real_ratings['rating']))   
    #remove real rating
    ratings = user_data[user_data['imdbID'] != imdbID] 
     
    print(ratings.to_numpy().size)
    print(ratings)
    
    if (ratings.to_numpy().size>0):   
        
        # Convert to Sparse Matrix Format
        #ratings_as_csr = csr_matrix(user_ratings)
        
        # Set algorithm and params 
        # TODO adapt neighbors based on number of given ratings by user_id
        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

        # Training
        print('---- Training ConBF-kNN-Algorithm ----')
        knn.fit(ratings)

        #Prediction -> get x nearest neighbors
        
        print("Find Movie row for kNN")
        print(user_data.loc[user_data['imdbID'] == imdbID])
    
        distances , indices = knn.kneighbors(user_data.loc[user_data['imdbID'] == imdbID], n_neighbors=k_neighbors+1)
        
        
        # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbours
        for i in range(0,indices.size):
            neighbors = ratings.iloc[[indices[0,i]]]
            
        
        #pred = mean(neighbors['rating'])
        
        pred = np.mean(neighbors['rating'])
        
        print('Predicted Rating for '+str(imdbID)+' :'+str(pred))
        print('Real Rating of '+str(imdbID)+' was'+ str(real_ratings['rating']))
        # return rating prediciton
        return pred
        
    else:
         return "User has not rated other movies. Check input"
    

In [65]:
user_ratings.head()

,index,imdbID,Year,Rated,Released,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,user_id,rating
5721,5721,112281.0,1995.0,2.0,11.0,90.0,6.4,196970.0,3.1,4.5,0.0,0.0,0.0,0.0,123.0,1.5
6693,6693,113627.0,1995.0,3.0,2.0,111.0,7.5,111148.0,9.0,8.2,1.0,0.0,0.0,0.0,123.0,5.0
7801,7801,114746.0,1995.0,3.0,1.0,129.0,8.0,563295.0,9.0,7.4,0.0,2.0,0.0,0.0,123.0,4.0
17242,17242,75314.0,1976.0,3.0,2.0,114.0,8.3,691354.0,9.5,9.4,0.0,4.0,0.0,0.0,123.0,5.0
18110,18110,145653.0,1999.0,3.0,1.0,145.0,7.3,19722.0,5.2,5.4,0.0,1.0,0.0,0.0,123.0,4.5


In [66]:
user_ratings.loc[user_ratings['imdbID']==113627]

,index,imdbID,Year,Rated,Released,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,user_id,rating
6693,6693,113627.0,1995.0,3.0,2.0,111.0,7.5,111148.0,9.0,8.2,1.0,0.0,0.0,0.0,123.0,5.0


In [67]:
user_ratings.dtypes

index                       int64
imdbID                    float64
Year                      float64
Rated                     float64
Released                  float64
Runtime                   float64
imdbRating                float64
imdbVotes                 float64
Rotten Tomatoes           float64
Metacritic                float64
Oscars_won                float64
Oscars_nominated          float64
Golden_globe_won          float64
Golden_globe_nominated    float64
user_id                   float64
rating                    float64
dtype: object

In [68]:
### Testing function ###
imdbID = 113627
# Aufpassen userID und imdbID als float übergeben!!
userID = 213
pred = predict_movie_rating(imdbID, userID, user_ratings)
print(pred)

         index    imdbID    Year  Rated  Released  Runtime  imdbRating  \
5721      5721  112281.0  1995.0    2.0      11.0     90.0         6.4   
6693      6693  113627.0  1995.0    3.0       2.0    111.0         7.5   
7801      7801  114746.0  1995.0    3.0       1.0    129.0         8.0   
17242    17242   75314.0  1976.0    3.0       2.0    114.0         8.3   
18110    18110  145653.0  1999.0    3.0       1.0    145.0         7.3   
...        ...       ...     ...    ...       ...      ...         ...   
700813  700813  212712.0  2004.0    3.0       9.0    129.0         7.4   
705597  705597  395169.0  2004.0    2.0       2.0    121.0         8.1   
741296  741296   75314.0  1976.0    3.0       2.0    114.0         8.3   
744482  744482  407265.0  2005.0    3.0       3.0    103.0         7.4   
761133  761133  449467.0  2006.0    3.0      11.0    143.0         7.4   

        imdbVotes  Rotten Tomatoes  Metacritic  Oscars_won  Oscars_nominated  \
5721     196970.0              

In [69]:
# TODO Remove a movie that was rated by a testUser and call function and check


In [70]:
# TODO: Perform PCA on result / Find out which features are responsible for the distances

In [ ]:
pca = PCA(**kwargs)
pca.fit(X_train)
pca.explained_variance_ratio_.round(5)

In [ ]:
#X_train_trans = pca.transform(X_train)[:, :4]
#X_test_trans = pca.transform(X_test)[:, :4]

In [ ]:
# TODO: Function schreiben, die statt dem mean() der Nachbarn mithilfe des Abstandes die ratings gewichtet
# Weiter entfernte filme werden geringer gewichtet nähere Filme werden stärker gewichtet